#### Import Modules

In [13]:
import os
import json
import time
import pandas as pd
import numpy as np
from yahoo_oauth import OAuth2
import yahoo_fantasy_api as yfa

#### Authorize via Oauth2

In [2]:
sc = OAuth2(None, None, from_file='C:\\Users\\phili\\OneDrive\\Documents\\DataProjects\\fantasyBaseballTools\\JupyterNotebooks\\oauth2.json')

[2020-09-02 12:53:41,502 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2020-09-02 12:53:41,516 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 470.3909854888916
[2020-09-02 12:53:41,520 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


#### Set parameters

In [3]:
# Set the game (e.g. mlb, nba, etc)
gameToUse = 'mlb'

# Set the league ID
leagueToUse = '136325'

# Final columns in the player tables
columnsToUse = ['player_id', 'name', 'eligible_positions']

# Output Folder PATH
outputFolder = r'C:\Users\phili\OneDrive\Documents\DataProjects\data'

# Create a timestamp
today = time.strftime('%Y%m%d')

#### Read-in Game and League settings for the Yahoo! Fantasy API

In [4]:
# Construct league code
leagueCode = gameToUse + '.l.' + leagueToUse

# Set-up game collection
gm = yfa.Game(sc, 'mlb')

# Set-up league...
lg = gm.to_league(leagueCode)

#### Read-in free agent, taken, and waivered players

In [5]:
##### Free Agents #####
print('...Downloading Yahoo! data')
# Batting
battingFAs = lg.free_agents('B')
battingFAs = pd.DataFrame(battingFAs)[columnsToUse]

# Pitching
pitchingFAs = lg.free_agents('P')
pitchingFAs = pd.DataFrame(pitchingFAs)[columnsToUse]

##### Taken #####
takenPlyrs = pd.DataFrame(lg.taken_players())[columnsToUse]

##### Waivered #####
waiverPlyrs = pd.DataFrame(lg.waivers())[columnsToUse]

##### My Team #####
myTeam = lg.to_team(leagueCode + '.t.1')
myPlyrs = pd.DataFrame(myTeam.roster())[columnsToUse]

...Downloading Yahoo! data


#### Combine all the dataframes

In [36]:
# Combine all dataframes
listOfAvail = [battingFAs, pitchingFAs, waiverPlyrs]
availablePlayers = pd.concat(listOfAvail)

# Rename columns
availablePlayers.rename(columns = {'player_id':'Player_ID','name':'Keeper', 'eligible_positions': 'Eligible_Positions'},
                        inplace = True)

takenPlyrs.rename(columns = {'player_id':'Player_ID','name':'Keeper', 'eligible_positions': 'Eligible_Positions'},
                        inplace = True)

myPlyrs.rename(columns = {'player_id':'Player_ID','name':'Keeper', 'eligible_positions': 'Eligible_Positions'},
                        inplace = True)

# Reorder columns
availablePlayers = availablePlayers[['Keeper', 'Player_ID', 'Eligible_Positions']]
takenPlyrs = takenPlyrs[['Keeper', 'Player_ID', 'Eligible_Positions']]
myPlyrs = myPlyrs[['Keeper', 'Player_ID', 'Eligible_Positions']]

# Assure Player_ID is a string
availablePlayers['Player_ID'] = availablePlayers['Player_ID'].astype('str')
takenPlyrs['Player_ID'] = takenPlyrs['Player_ID'].astype('str')
myPlyrs['Player_ID'] = myPlyrs['Player_ID'].astype('str')

# Construct file paths
availableFilename = 'totalAvailableList_' + today + '.csv'
takenFilename = 'totalKeeperList_' + today + '.csv'
myTeamFilename = 'MyTeam.csv'

# Write to csv
print('...writing out data to data: ' + availableFilename + ', ' + takenFilename + ' and ' + myTeamFilename + ' in ' + outputFolder)
availablePlayers.to_csv(os.path.join(outputFolder, availableFilename), encoding='utf-8-sig', index = False)
takenPlyrs.to_csv(os.path.join(outputFolder, takenFilename), encoding='utf-8-sig', index = False)
myPlyrs.to_csv(os.path.join(outputFolder, myTeamFilename), encoding='utf-8-sig', index = False)

...writing out data to data: totalAvailableList_20200902.csv, totalKeeperList_20200902.csv and MyTeam.csv in C:\Users\phili\OneDrive\Documents\DataProjects\data
